# 🎵 이벤트 추천 시스템 데모 – 콘텐츠 기반 필터링
이 노트북은 **`merged_events_clean.csv`** 데이터 한 파일로 다음 과정을 모두 시연합니다.

1. 데이터 로드 및 기초 정제
2. TF‑IDF + 메타데이터 벡터화
3. 코사인 거리 기반 이웃 탐색 모델 학습
4. `recommend()` 함수로 간단 추천
5. (선택) FastAPI 엔드포인트 제공

## 0. 라이브러리 임포트 및 설정

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import hstack
import joblib

print('라이브러리 불러오기 완료')

라이브러리 불러오기 완료


## 1. 데이터 불러오기 & 확인

In [2]:
CSV_PATH = Path('merged_events_clean - merged_events_clean.csv')  # 필요 시 경로 수정
df = pd.read_csv(CSV_PATH)

print('행/열 수:', df.shape)
df.head()

행/열 수: (3988, 8)


,link,content,place,date,time,in advance,cover,image
0,https://www.instagram.com/_a_c_s___/p/DJdZN_oJ...,your arms are my cocoon & godfuck Korea Tour 2...,서울 중구 수표로6길 10 지하1층,2025.7.4,18:30,35000,45000,NaN
1,https://www.instagram.com/_a_c_s___/p/DJvQpH6z...,Dayoung과 Mokhzolla 수강생들의 수료 파티!\n\n다들 첫 긱 때의 그...,서울 중구 수표로6길 10 지하1층,NaN,20:00,10000,NaN,NaN
2,https://www.instagram.com/_a_c_s___/p/DIQeZOIJ...,[2025.4.27] “돌진 / 突進” w/ Ryosuke Kiyasu\n\n현재도...,서울 중구 수표로6길 10 지하1층,2025.4.27,18:00,20000,25000,NaN
3,https://www.instagram.com/_a_c_s___/p/DIQAIfrz...,"움직임\n\n모든 것은 움직임이죠. 몸도, 시간도, 소리도, 마음도…. 현재 아시아...",서울 중구 수표로6길 10 지하1층,NaN,19:00,20000,NaN,NaN
4,https://www.instagram.com/_a_c_s___/p/DIJglNIz...,2025/4/11(Fri) 8PM-\nKOMAoto in South Korea\nA...,서울 중구 수표로6길 10 지하1층,NaN,20:00,15000,NaN,NaN


## 2. 기본 전처리
* 날짜·시간 파싱
* 가격 결측값 보정
* 위치(시/구) 추출

In [6]:
# 날짜/시간
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['time'] = pd.to_datetime(df['time'], format='%H:%M', errors='coerce').dt.time

# 가격 컬럼 숫자형 변환
df['price_adv']  = pd.to_numeric(df['in advance'], errors='coerce')
df['price_door'] = pd.to_numeric(df['cover'], errors='coerce')

mid_price = df[['price_adv', 'price_door']].stack().median()
df['price_adv'].fillna(df['price_door'], inplace=True)
df['price_door'].fillna(df['price_adv'], inplace=True)
df[['price_adv','price_door']] = df[['price_adv','price_door']].fillna(mid_price)

# 위치(시/구) 추출
loc = df['place'].fillna('').str.extract(r'^(?P<city>[^ ]+)\s*(?P<gu>[^ ]+)?')
df['loc_sigu'] = loc['city'].fillna('') + ' ' + loc['gu'].fillna('')
df.loc[df['loc_sigu'].str.strip()=='','loc_sigu'] = 'unknown'

df[['date','time','price_adv','price_door','loc_sigu']].head()

/var/folders/bn/4pkyl36d075dfzp1bl61d37h0000gn/T/ipykernel_40588/3925682135.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price_adv'].fillna(df['price_door'], inplace=True)
/var/folders/bn/4pkyl36d075dfzp1bl61d37h0000gn/T/ipykernel_40588/3925682135.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting val

,date,time,price_adv,price_door,loc_sigu
0,2025-07-04,NaT,35000.0,45000.0,서울 중구
1,NaT,NaT,10000.0,10000.0,서울 중구
2,2025-04-27,NaT,20000.0,25000.0,서울 중구
3,NaT,NaT,20000.0,20000.0,서울 중구
4,NaT,NaT,15000.0,15000.0,서울 중구


## 3. 피처 엔지니어링
### 3‑1. TF‑IDF(텍스트) 벡터화

In [7]:
text_corpus = (
    df['content'].fillna('') + ' ' +
    df['place'].fillna('')   + ' ' +
    df['loc_sigu'].fillna('')
)

tfidf = TfidfVectorizer(max_features=10_000,
                        ngram_range=(1,2),
                        min_df=3,
                        stop_words='english')
X_text = tfidf.fit_transform(text_corpus)
X_text

<3988x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 243130 stored elements in Compressed Sparse Row format>

### 3‑2. 숫자형 & 범주형 메타데이터 인코딩

In [8]:
num_cols = ['price_adv', 'price_door']
cat_cols = ['loc_sigu']

pre = ColumnTransformer([
    ('num', MinMaxScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

X_meta = pre.fit_transform(df)
X_meta

<3988x18 sparse matrix of type '<class 'numpy.float64'>'
	with 11331 stored elements in Compressed Sparse Row format>

### 3‑3. 희소 행렬 결합

In [9]:
X_all = hstack([X_text, X_meta]).tocsr()
X_all

<3988x10018 sparse matrix of type '<class 'numpy.float64'>'
	with 254461 stored elements in Compressed Sparse Row format>

## 4. 코사인 Nearest Neighbors 모델 학습 및 저장

In [10]:
knn = NearestNeighbors(metric='cosine', n_neighbors=20, n_jobs=-1)
knn.fit(X_all)

MODEL_DIR = Path('model')
MODEL_DIR.mkdir(exist_ok=True)
joblib.dump({'tfidf': tfidf, 'pre': pre, 'knn': knn, 'df': df}, MODEL_DIR/'recommender_ko.joblib')
print('모델 저장 완료:', MODEL_DIR/'recommender_ko.joblib')

모델 저장 완료: model/recommender_ko.joblib


## 5. 추천 함수 정의

In [11]:
job = joblib.load(MODEL_DIR/'recommender_ko.joblib')

def encode_query(q: dict):
    """사용자 입력(dict) → 모델 입력 벡터"""
    txt_vec = job['tfidf'].transform([q.get('keywords','')])
    meta_df = pd.DataFrame([{
        'price_adv':  q.get('price_max', mid_price),
        'price_door': q.get('price_max', mid_price),
        'loc_sigu':   q.get('location', 'unknown')
    }])
    meta_vec = job['pre'].transform(meta_df)
    return hstack([txt_vec, meta_vec])

def recommend(query: dict, top_k=5):
    q_vec = encode_query(query)
    dist, idx = job['knn'].kneighbors(q_vec, n_neighbors=top_k)
    recs = job['df'].iloc[idx[0]].copy()
    recs['score'] = 1 - dist[0]  # 코사인 유사도
    return recs[['link','content','place','date','time','price_adv','price_door','score']]


## 6. 동작 예시

In [12]:
sample = {
    'keywords': 'psychedelic rock live',
    'price_max': 35000,
    'location': '서울 마포구'
}
recommend(sample)

,link,content,place,date,time,price_adv,price_door,score
53,https://www.instagram.com/space_hangang/p/CvFF...,<남경운과 서울 사람들 Vol.1>\n\nTicket Link in @space_h...,서울 마포구 와우산로 128 지하1층),2023-08-06,NaT,33000.0,33000.0,0.525651
706,https://www.instagram.com/hermit_kr/p/C_ryP36S...,안녕하세요 여러분!\n\n적바림JeokVaRim 앨범 “U R NOT EXTRA”\...,서울 마포구 양화로6길 27 지하 1층,2025-09-14,NaT,25000.0,10000.0,0.518624
703,https://www.instagram.com/hermit_kr/p/DB1aTTHS...,예매 문의 @bonui777\n\n[리프 데뷔 1주년 기념 공연] @lifficia...,서울 마포구 양화로6길 27 지하 1층,2024-11-15,NaT,25000.0,10000.0,0.513847
707,https://www.instagram.com/hermit_kr/p/C_rsxQkS...,"[2024 Cécile(세실) 생일잔치]\n\n안녕하세요, 더픽쳐스의 드러머 조민영...",서울 마포구 양화로6길 27 지하 1층,2024-09-12,NaT,25000.0,10000.0,0.513847
773,https://www.instagram.com/hermit_kr/p/Cu0l6Sjv...,.\n📌 7/21 <금요일 밤 12시 공연> 라인업\n\n현재 저희 레몬 라이브 하...,서울 마포구 양화로6길 27 지하 1층,2023-07-21,NaT,20000.0,10000.0,0.510019


## 7. (선택) FastAPI 엔드포인트 예시

In [13]:
"""이 셀을 api.py 파일로 저장한 뒤 아래 명령어로 실행하세요.

    uvicorn api:app --host 0.0.0.0 --port 8000
"""
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()
model = joblib.load('model/recommender_ko.joblib')

class Query(BaseModel):
    keywords: str = ''
    price_max: float | None = None
    location: str = ''

@app.post('/recommend')
def rec_api(q: Query, top_k:int=5):
    res = recommend(q.dict(), top_k=top_k)
    return res.to_dict(orient='records')
# if __name__ == '__main__':
#     uvicorn.run(app, host='0.0.0.0', port=8000)

ModuleNotFoundError: No module named 'fastapi'